### Install the required libraries

In [1]:
!pip install praw --user
!pip install numpy --user
!pip install pandas --user
!pip install lxml --user

In [2]:
import praw 
from praw.models import MoreComments
import numpy as np
import re
import pandas as pd
from bs4 import BeautifulSoup
import string

### Collect Submissions for each flair

In [3]:
flairs_mapping = {'AskIndia':1,'Business/Finance':2,'Food':3,'Non-Political':4,'Photography':5,'Policy/Economy':6,'Politics':7,'Scheduled':8,'Science/Technology':9,'Sports':10}
flairs = flairs_mapping.keys()
reddit = praw.Reddit(client_id="z8UhRRiFnEVZ8Q",
                     client_secret="bV7OxwG-VKjdyKxcuUihnV1lNPg",
                     password="qazwsx123",
                     username="lshreyasharmal",
                     user_agent="bot1 user agent")

subreddit = reddit.subreddit("india")
submissions = {}
for flair in flairs:
    get_subreddits = subreddit.search(flair, limit=200)
    for submission in get_subreddits:
        if flair not in submissions.keys():
            submissions[flair] = []
        submissions[flair].append(submission)
np.save("submissions.npy",submissions)

### Collect post features from submissions

In [9]:
dataset = []
for flair_value,subs in submissions.items():
    for submission in subs:
        post = vars(submission)
        submission.comments.replace_more(limit = None)
        comments = ""
        for comment in submission.comments:
            comments += " " + comment.body
        post["comments"] = comments
        author = str(submission.author).split("'")
        post['author'] = author[len(author)-2]
        post['target'] = flairs_mapping[flair_value]
        dataset.append(post)
np.save("dataset.npy",dataset)
data = pd.DataFrame(dataset)

### Drop Columns with same value accross all rows

In [12]:
cols_to_drop = []
for col in data.columns:
    try:
        if data[col].nunique()<2:
            cols_to_drop.append(col)
    except:
        cols_to_drop.append(col)
data = data.drop(cols_to_drop,axis=1)

### Drop Flair Type column from data

In [13]:
data = data.drop(['link_flair_text'],axis=1)

### Preprocess different data types

In [14]:
numerical_cols = [col for col in data.columns if np.issubdtype(data[col].dtype, np.number)]

boolean_cols = [col for col in data.columns if data[col].dtype == "bool"]

object_cols = [col for col in data.columns if data[col].dtype == "object"]

def preprocess_strings(x):
    x = re.sub(r'http\S+', '', x, flags=re.MULTILINE)
    x = BeautifulSoup(x, "html.parser").text
    x = x.translate(str.maketrans("","",string.punctuation))
    x = re.sub(r'[^a-zA-z\s]', '', x)
    x = x.lower()
    x = x.strip()
    x = re.sub(r'\W'," ",x)
    return x

final_data = []

for row in data.iterrows():
    final_data_point = {}
    for col in object_cols:
        if isinstance(row[1][col],str):
            final_data_point[col] = preprocess_strings(row[1][col])
    for col in boolean_cols:
        final_data_point[col] = 1 if row[1][col] else 0
    for col in numerical_cols:
        final_data_point[col] = row[1][col]
    final_data.append(final_data_point)
    
final_dataframe = pd.DataFrame(final_data)

In [15]:
final_dataframe.to_csv("final_data.csv",index=True,header=True)

In [17]:
np.save("string_cols.npy",object_cols)
np.save("numerical_cols.npy",numerical_cols)
np.save("boolean_cols.npy",boolean_cols)

In [18]:
final_dataframe.columns

Index(['allow_live_comments', 'archived', 'author',
       'author_flair_background_color', 'author_flair_css_class',
       'author_flair_template_id', 'author_flair_text',
       'author_flair_text_color', 'author_flair_type', 'author_fullname',
       'author_premium', 'comments', 'contest_mode', 'created', 'created_utc',
       'domain', 'gilded', 'hide_score', 'id', 'is_original_content',
       'is_reddit_media_domain', 'is_self', 'is_video',
       'link_flair_background_color', 'link_flair_css_class',
       'link_flair_template_id', 'link_flair_text_color', 'link_flair_type',
       'locked', 'name', 'no_follow', 'num_comments', 'num_crossposts',
       'num_duplicates', 'permalink', 'post_hint', 'score', 'selftext',
       'selftext_html', 'send_replies', 'spoiler', 'stickied',
       'subreddit_subscribers', 'suggested_sort', 'target', 'thumbnail',
       'thumbnail_height', 'thumbnail_width', 'title', 'total_awards_received',
       'ups', 'upvote_ratio', 'url', 'whitelist_